# STAT 451 Final Project - Group 12 (11:00am)
Mariya Siddiqui, Gavin Fring, Nicholas Dubois, Fiena Sapari, Ahmad Latiffi

## Data Preparation, Exploration, and Transformation
In this section, we do the following:

 1. Load raw data (which is stored locally at `./datasets/airline_delays.csv`)
 2. Run a rough profile of it to get an idea of what the dataset looks like as a whole, and on a columnar level. The profile numerical type columns (floats, ints, etc.) will also contain basic statistics about the values and distribution of that column. For categorical columns, these values are skipped. 
 3. Clean and prep the raw data to be consumable by our models

In [1]:
# Imports
import pandas as pd
import random

# Suppressing deprecation warnings temporarily as they take up a lot of room in output
import warnings
warnings.filterwarnings('ignore')

### 1. Load Data

In [2]:
raw_df = pd.read_csv("./datasets/raw_airline_delays.csv")
raw_df.head(5)

,year,month,carrier,carrier_name,airport,airport_name,arr_flights,arr_del15,carrier_ct,weather_ct,...,security_ct,late_aircraft_ct,arr_cancelled,arr_diverted,arr_delay,carrier_delay,weather_delay,nas_delay,security_delay,late_aircraft_delay
0,2023,8,9E,Endeavor Air Inc.,ABE,"Allentown/Bethlehem/Easton, PA: Lehigh Valley ...",89.0,13.0,2.25,1.60,...,0.0,5.99,2.0,1.0,1375.0,71.0,761.0,118.0,0.0,425.0
1,2023,8,9E,Endeavor Air Inc.,ABY,"Albany, GA: Southwest Georgia Regional",62.0,10.0,1.97,0.04,...,0.0,7.42,0.0,1.0,799.0,218.0,1.0,62.0,0.0,518.0
2,2023,8,9E,Endeavor Air Inc.,AEX,"Alexandria, LA: Alexandria International",62.0,10.0,2.73,1.18,...,0.0,4.28,1.0,0.0,766.0,56.0,188.0,78.0,0.0,444.0
3,2023,8,9E,Endeavor Air Inc.,AGS,"Augusta, GA: Augusta Regional at Bush Field",66.0,12.0,3.69,2.27,...,0.0,1.57,1.0,1.0,1397.0,471.0,320.0,388.0,0.0,218.0
4,2023,8,9E,Endeavor Air Inc.,ALB,"Albany, NY: Albany International",92.0,22.0,7.76,0.00,...,0.0,11.28,2.0,0.0,1530.0,628.0,0.0,134.0,0.0,768.0


### 2. Profile Raw Data

In [3]:
def dataframe_profile(df):
    '''
    This function creates a profile of a particular dataset given in the form of a dataframe. 
    It outputs a dataframe that contains the profile. 
    '''
    # Create a DataFrame to store the profile
    profile_df = pd.DataFrame(
        columns=[
            "Column",
            "Data Type",
            "Missing Values",
            "Unique Values",
            "Top Value",
            "Frequency",
            "Min",
            "25th Percentile",
            "Median",
            "75th Percentile",
            "Max",
            "Mean",
            "Standard Deviation",
        ]
    )

    # Populate the profile DataFrame
    for column in df.columns:
        data_type = df[column].dtype
        missing_values = df[column].isnull().sum()
        unique_values = df[column].nunique()
        top_value = df[column].mode().iloc[0] if unique_values > 0 else None
        frequency = df[column].value_counts().sort_values(ascending=False).iloc[0] if unique_values > 0 else None

        # Additional metadata for numeric columns
        if pd.api.types.is_numeric_dtype(df[column]):
            min_value = df[column].min()
            percentile_25 = df[column].quantile(0.25)
            median_value = df[column].median()
            percentile_75 = df[column].quantile(0.75)
            max_value = df[column].max()
            mean_value = df[column].mean()
            std_deviation = df[column].std()

        # No additional metadata for other types
        else:
            min_value = None
            percentile_25 = None
            median_value = None
            percentile_75 = None
            max_value = None

            mean_value = None
            std_deviation = None

        # Use loc to add rows to the DataFrame
        profile_df.loc[len(profile_df)] = {
            "Column": column,
            "Data Type": data_type,
            "Missing Values": missing_values,
            "Unique Values": unique_values,
            "Top Value": top_value,
            "Frequency": frequency,
            "Min": min_value,
            "25th Percentile": percentile_25,
            "Median": median_value,
            "75th Percentile": percentile_75,
            "Max": max_value,
            "Mean": mean_value,
            "Standard Deviation": std_deviation,
        }

    # Summary stats 
    (df_rows, df_columns) = df.shape
    print(f"The dataframe has {df_rows} rows and {df_columns} columns")

    return profile_df

In [4]:
profile_raw_df = dataframe_profile(raw_df)
display(profile_raw_df)

The dataframe has 345323 rows and 21 columns


,Column,Data Type,Missing Values,Unique Values,Top Value,Frequency,Min,25th Percentile,Median,75th Percentile,Max,Mean,Standard Deviation
0,year,int64,0,21,2019,20946,2003,2008.00,2013.00,2019.00,2023,2013.206213,6.042778
1,month,int64,0,12,6,30098,1,4.00,7.00,9.00,12,6.493312,3.431955
2,carrier,object,0,29,OO,42164,None,NaN,NaN,NaN,None,NaN,NaN
3,carrier_name,object,0,33,SkyWest Airlines Inc.,42164,None,NaN,NaN,NaN,None,NaN,NaN
4,airport,object,0,420,DTW,3243,None,NaN,NaN,NaN,None,NaN,NaN
5,airport_name,object,0,444,"Detroit, MI: Detroit Metro Wayne County",3243,None,NaN,NaN,NaN,None,NaN,NaN
6,arr_flights,float64,509,7456,31.0,9864,1.0,58.00,120.00,270.00,21977.0,378.935876,1021.719103
7,arr_del15,float64,747,2366,0.0,9683,0.0,9.00,22.00,56.00,6377.0,73.002383,199.130487
8,carrier_ct,float64,509,19260,0.0,18747,0.0,3.00,8.15,19.76,1792.07,21.416112,48.984100
9,weather_ct,float64,509,5766,0.0,136811,0.0,0.00,0.60,2.01,717.94,2.633833,9.906200


### 3. Refine raw data
#### Categorical columns
The categorical columns `carrier`, `carrier_name`, `airport`, and `airport_name` come in pairs. It would make sense that each pair would have the same number of unique values, but since the `_name` columns have more in both cases, its clear that some `carrier` and `airport` values correspond to more than one possible naming of the location. 

Because of this naming inconsistency, we choose to use the identifying columns `airport` and `carrier` as features to consider instead of the names.

In [5]:
df_carriers = raw_df[["carrier", "carrier_name"]]
df_carriers_cts = df_carriers.value_counts( sort=False).to_frame()
display(df_carriers_cts)

count
carrier carrier_name                       
9E      Endeavor Air Inc.              7593
        Pinnacle Airlines Inc.         6454
AA      American Airlines Inc.        21591
AQ      Aloha Airlines Inc.             253
AS      Alaska Airlines Inc.          14220
B6      JetBlue Airways               12573
CO      Continental Air Lines Inc.     6917
DH      Atlantic Coast Airlines        1253
        Independence Air                671
DL      Delta Air Lines Inc.          29690
EV      Atlantic Southeast Airlines   12204
        ExpressJet Airlines Inc.      13159
        ExpressJet Airlines LLC        1436
F9      Frontier Airlines Inc.        13568
FL      AirTran Airways Corporation    6956
G4      Allegiant Air                  8280
HA      Hawaiian Airlines Inc.         3881
HP      America West Airlines Inc.     1581
MQ      American Eagle Airlines Inc.  15409
        Envoy Air                     11886
NK      Spirit Air Lines               4698
NW      Northwest Airlines Inc.        8084
OH      Comair Inc.                    7775
        PSA Airlines Inc.              6190
OO      SkyWest Airlines Inc.         42164
QX      Horizon Air                    1157
RU      ExpressJet Airlines Inc.       4136
TZ      ATA Airlines d/b/a ATA          924
UA      United Air Lines Inc.         20794
US      US Airways Inc.               10698
VX      Virgin America                 1428
WN      Southwest Airlines Co.        19181
XE      ExpressJet Airlines Inc.       7375
YV      Mesa Airlines Inc.            15417
YX      Republic Airline               5727

In [6]:
df_airports = raw_df[["airport", "airport_name"]]
df_airports_cts = df_airports.value_counts( sort=False).to_frame()
display(df_airports_cts)

,,count
airport,airport_name,
ABE,"Allentown/Bethlehem/Easton, PA: Lehigh Valley International",1160
ABI,"Abilene, TX: Abilene Regional",286
ABQ,"Albuquerque, NM: Albuquerque International Sunport",2452
ABR,"Aberdeen, SD: Aberdeen Regional",140
ABY,"Albany, GA: Southwest Georgia Regional",247
...,...,...
XWA,"Williston, ND: Williston Basin International",47
YAK,"Yakutat, AK: Yakutat Airport",243
YKM,"Yakima, WA: Yakima Air Terminal/McAllister Field",39


In [7]:
refined_df = raw_df.drop(["carrier_name", "airport_name"], axis=1)
refined_df.head()

,year,month,carrier,airport,arr_flights,arr_del15,carrier_ct,weather_ct,nas_ct,security_ct,late_aircraft_ct,arr_cancelled,arr_diverted,arr_delay,carrier_delay,weather_delay,nas_delay,security_delay,late_aircraft_delay
0,2023,8,9E,ABE,89.0,13.0,2.25,1.60,3.16,0.0,5.99,2.0,1.0,1375.0,71.0,761.0,118.0,0.0,425.0
1,2023,8,9E,ABY,62.0,10.0,1.97,0.04,0.57,0.0,7.42,0.0,1.0,799.0,218.0,1.0,62.0,0.0,518.0
2,2023,8,9E,AEX,62.0,10.0,2.73,1.18,1.80,0.0,4.28,1.0,0.0,766.0,56.0,188.0,78.0,0.0,444.0
3,2023,8,9E,AGS,66.0,12.0,3.69,2.27,4.47,0.0,1.57,1.0,1.0,1397.0,471.0,320.0,388.0,0.0,218.0
4,2023,8,9E,ALB,92.0,22.0,7.76,0.00,2.96,0.0,11.28,2.0,0.0,1530.0,628.0,0.0,134.0,0.0,768.0


#### Missing values
Some of the numerical columns have missing values. Running a profile on the rows with missing values tells us that 
 1. the missing values aren't very strongly associated with any particular feature values
 2. the number of samples with missing values (747) is *very* small compared to the total number of samples in the data set

So, simply dropping the missing rows will suffice to prep the data set

In [8]:
missing_rows_df = refined_df[refined_df.isna().any(axis=1)]

profile_missing_rows_df = dataframe_profile(missing_rows_df)
display(profile_missing_rows_df)

The dataframe has 747 rows and 19 columns


,Column,Data Type,Missing Values,Unique Values,Top Value,Frequency,Min,25th Percentile,Median,75th Percentile,Max,Mean,Standard Deviation
0,year,int64,0,21,2020,260,2003,2009.0,2016.0,2020.00,2023,2014.693440,5.873875
1,month,int64,0,12,4,195,1,4.0,5.0,9.00,12,6.283802,3.199328
2,carrier,object,0,26,OO,108,None,NaN,NaN,NaN,None,NaN,NaN
3,airport,object,0,239,PVD,10,None,NaN,NaN,NaN,None,NaN,NaN
4,arr_flights,float64,509,38,1.0,98,1.0,1.0,2.0,8.75,120.0,8.689076,16.751975
5,arr_del15,float64,747,0,None,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,carrier_ct,float64,509,1,0.0,238,0.0,0.0,0.0,0.00,0.0,0.000000,0.000000
7,weather_ct,float64,509,1,0.0,238,0.0,0.0,0.0,0.00,0.0,0.000000,0.000000
8,nas_ct,float64,509,1,0.0,238,0.0,0.0,0.0,0.00,0.0,0.000000,0.000000
9,security_ct,float64,509,1,0.0,238,0.0,0.0,0.0,0.00,0.0,0.000000,0.000000


In [9]:
df_carrier_counts = refined_df[["carrier"]].value_counts().to_frame()
df_missing_carrier_counts = missing_rows_df[["carrier"]].value_counts().to_frame()
df_joined_carrier_counts = df_carrier_counts.join(df_missing_carrier_counts, how="outer", lsuffix="_raw", rsuffix="_missing")
df_joined_carrier_counts = df_joined_carrier_counts.assign(prop_missing=df_joined_carrier_counts["count_missing"] / df_joined_carrier_counts["count_raw"])
display(df_joined_carrier_counts[df_joined_carrier_counts.notnull().all(axis=1)])

,count_raw,count_missing,prop_missing
carrier,,,
9E,14047,54.0,0.003844
AA,21591,11.0,0.000509
AS,14220,2.0,0.000141
B6,12573,8.0,0.000636
CO,6917,9.0,0.001301
DH,1924,1.0,0.000520
DL,29690,65.0,0.002189
EV,26799,103.0,0.003843
F9,13568,32.0,0.002358


In [10]:
df_airport_counts = refined_df[["airport"]].value_counts().to_frame()
df_missing_airport_counts = missing_rows_df[["airport"]].value_counts().to_frame()
df_joined_airport_counts = df_airport_counts.join(df_missing_airport_counts, how="outer", lsuffix="_raw", rsuffix="_missing")
df_joined_airport_counts = df_joined_airport_counts.assign(prop_missing=df_joined_airport_counts["count_missing"] / df_joined_airport_counts["count_raw"])
display(df_joined_airport_counts[df_joined_airport_counts.notnull().all(axis=1)])

,count_raw,count_missing,prop_missing
airport,,,
ABE,1160,3.0,0.002586
ABI,286,4.0,0.013986
ABQ,2452,4.0,0.001631
ABY,247,2.0,0.008097
ACY,229,9.0,0.039301
...,...,...,...
TYS,1623,1.0,0.000616
VCT,61,1.0,0.016393
VLD,244,1.0,0.004098


In [11]:
refined_df = refined_df.dropna(axis=0, how="any", ignore_index=True)
refined_df.head()

,year,month,carrier,airport,arr_flights,arr_del15,carrier_ct,weather_ct,nas_ct,security_ct,late_aircraft_ct,arr_cancelled,arr_diverted,arr_delay,carrier_delay,weather_delay,nas_delay,security_delay,late_aircraft_delay
0,2023,8,9E,ABE,89.0,13.0,2.25,1.60,3.16,0.0,5.99,2.0,1.0,1375.0,71.0,761.0,118.0,0.0,425.0
1,2023,8,9E,ABY,62.0,10.0,1.97,0.04,0.57,0.0,7.42,0.0,1.0,799.0,218.0,1.0,62.0,0.0,518.0
2,2023,8,9E,AEX,62.0,10.0,2.73,1.18,1.80,0.0,4.28,1.0,0.0,766.0,56.0,188.0,78.0,0.0,444.0
3,2023,8,9E,AGS,66.0,12.0,3.69,2.27,4.47,0.0,1.57,1.0,1.0,1397.0,471.0,320.0,388.0,0.0,218.0
4,2023,8,9E,ALB,92.0,22.0,7.76,0.00,2.96,0.0,11.28,2.0,0.0,1530.0,628.0,0.0,134.0,0.0,768.0


#### Re-profile refined data
and save to csv for easy usage in models

In [12]:
profile_refined_df = dataframe_profile(refined_df)
display(profile_refined_df)

The dataframe has 344576 rows and 19 columns


,Column,Data Type,Missing Values,Unique Values,Top Value,Frequency,Min,25th Percentile,Median,75th Percentile,Max,Mean,Standard Deviation
0,year,int64,0,21,2019,20927,2003,2008.00,2013.00,2019.00,2023,2013.202989,6.042750
1,month,int64,0,12,6,30045,1,4.00,7.00,9.00,12,6.493766,3.432433
2,carrier,object,0,29,OO,42056,None,NaN,NaN,NaN,None,NaN,NaN
3,airport,object,0,418,DTW,3243,None,NaN,NaN,NaN,None,NaN,NaN
4,arr_flights,float64,0,7456,31.0,9863,1.0,58.00,120.00,270.00,21977.0,379.191606,1022.025448
5,arr_del15,float64,0,2366,0.0,9683,0.0,9.00,22.00,56.00,6377.0,73.002383,199.130487
6,carrier_ct,float64,0,19260,0.0,18509,0.0,3.00,8.16,19.77,1792.07,21.430904,48.997779
7,weather_ct,float64,0,5766,0.0,136573,0.0,0.00,0.60,2.02,717.94,2.635652,9.909379
8,nas_ct,float64,0,21760,0.0,33952,-0.01,1.61,5.35,15.08,4091.27,23.598193,83.292069
9,security_ct,float64,0,1186,0.0,298435,0.0,0.00,0.00,0.00,80.56,0.181815,0.854187


In [13]:
refined_df.to_csv("./datasets/refined_airline_delays.csv", index=False)